In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
orders_df = pd.read_csv("../data/raw/orders.csv")
op_prior_df = pd.read_csv("../data/raw/order_products__prior.csv")

In [3]:
prior_info_df = pd.merge(op_prior_df, orders_df, how = "inner", on = "order_id")

In [4]:
prior_info_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,202279,prior,3,5,9,8.0
1,2,28985,2,1,202279,prior,3,5,9,8.0
2,2,9327,3,0,202279,prior,3,5,9,8.0
3,2,45918,4,1,202279,prior,3,5,9,8.0
4,2,30035,5,0,202279,prior,3,5,9,8.0


In [17]:
uid_pid_info = prior_info_df[['user_id', 'product_id']].groupby(['user_id','product_id']).size()

In [18]:
uid_pid_info = uid_pid_info.reset_index()

In [22]:
uid_pid_info.rename(columns = {0:'count'}, inplace=True)

In [23]:
uid_pid_info.head()

,user_id,product_id,count
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


In [35]:
uid_pid_info.user_id.max()

206209

In [34]:
uid_pid_info.product_id.max()

49688

In [33]:
uid_pid_info['count'].max()

99

In [36]:
uid_pid_info.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13307953 entries, 0 to 13307952
Data columns (total 3 columns):
user_id       int64
product_id    int64
count         int64
dtypes: int64(3)
memory usage: 304.6 MB


In [38]:
# reduce memory
uid_pid_info.user_id = uid_pid_info.user_id.astype('uint32')
uid_pid_info.product_id = uid_pid_info.product_id.astype('uint16')
uid_pid_info['count'] = uid_pid_info['count'].astype('uint8')

In [25]:
uid_pid_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13307953 entries, 0 to 13307952
Data columns (total 3 columns):
user_id       int64
product_id    int64
count         int64
dtypes: int64(3)
memory usage: 304.6 MB


In [39]:
uid_pid_info.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13307953 entries, 0 to 13307952
Data columns (total 3 columns):
user_id       uint32
product_id    uint16
count         uint8
dtypes: uint16(1), uint32(1), uint8(1)
memory usage: 88.8 MB


In [42]:
uid_pid_info.head()

,user_id,product_id,count
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


In [43]:
# dictionary format - {user_id : {product_id: count}}
uid_pid_cnt_dict = {k : dict(zip(g['product_id'], g['count'])) for k,g in uid_pid_info.groupby('user_id')}

In [44]:
len(uid_pid_cnt_dict.keys())

206209

In [52]:
uid_pid_cnt_dict[1]

{196: 10,
 10258: 9,
 10326: 1,
 12427: 10,
 13032: 3,
 13176: 2,
 14084: 1,
 17122: 1,
 25133: 8,
 26088: 2,
 26405: 2,
 30450: 1,
 35951: 1,
 38928: 1,
 39657: 1,
 41787: 1,
 46149: 3,
 49235: 2}

In [55]:
# weighted count
uid_pid_cnt_weighted_dict = {}
for k, g in uid_pid_cnt_dict.items():
    gs = sum(uid_pid_cnt_dict[k].values())
    uid_pid_cnt_weighted_dict[k] = {p_i: round(g_i/gs,5) for p_i, g_i in g.items()}

In [56]:
uid_pid_cnt_weighted_dict[1]

{196: 0.16949,
 10258: 0.15254000000000001,
 10326: 0.01695,
 12427: 0.16949,
 13032: 0.050849999999999999,
 13176: 0.0339,
 14084: 0.01695,
 17122: 0.01695,
 25133: 0.13558999999999999,
 26088: 0.0339,
 26405: 0.0339,
 30450: 0.01695,
 35951: 0.01695,
 38928: 0.01695,
 39657: 0.01695,
 41787: 0.01695,
 46149: 0.050849999999999999,
 49235: 0.0339}